# ANN- Univariate Time Series Solar Radiation(GHI) Forecasting with MLP, CNN, and LSTM Networks

# MLP

In [1]:
import torch

# Check if GPU is available
print("CUDA Available:", torch.cuda.is_available())

# Check the name of the GPU
if torch.cuda.is_available():
    print("GPU Name:", torch.cuda.get_device_name(0))


CUDA Available: True
GPU Name: Tesla T4


In [2]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118


Looking in indexes: https://download.pytorch.org/whl/cu118


In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
import subprocess
import time
import os

# Function to monitor power usage with nvidia-smi
def monitor_power(output_file):
    # Run nvidia-smi in the background to log power usage
    command = f"nvidia-smi --query-gpu=power.draw --format=csv -l 1 > {output_file}"
    process = subprocess.Popen(command, shell=True)
    return process

# Function to calculate energy consumption from the log
def calculate_energy_consumption(log_file):
    # Load the power log file
    data = pd.read_csv(log_file, skiprows=1, names=["Power (W)"])

    # Debug: Check log content
    print(f"Log file content:\n{data.head()}")

    # Clean the "Power (W)" column to remove non-numeric characters (e.g., " W")
    data["Power (W)"] = data["Power (W)"].str.extract(r'(\d+\.\d+|\d+)').astype(float)

    # Debug: Check cleaned data
    print(f"Cleaned log data:\n{data.head()}")

    # Calculate total energy in Joules (assuming logs are every 1 second)
    energy_joules = data["Power (W)"].sum()  # Energy = Power × Time (1 second per log)

    # Convert total energy to kWh
    energy_kwh = energy_joules / (3600 * 1000)

    return energy_joules, energy_kwh


# Add this at the beginning of your script or before starting power monitoring
def clear_log_file(log_file):
    if os.path.exists(log_file):
        os.remove(log_file)
        print(f"Cleared existing log file: {log_file}")

# Load and preprocess the dataset
data = pd.read_csv('jordan_pv.csv', parse_dates=['date'], index_col='date')
#data = pd.read_csv('Palestine-PV.csv', parse_dates=['date'], index_col='date')

data.index = pd.to_datetime(data.index)
data = data.sort_index()
data = data.iloc[:, 5].dropna().astype('float32')

# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data.values.reshape(-1, 1))

# Split data into train and test sets
train_size = int(len(scaled_data) * 0.80)
train, test = scaled_data[:train_size], scaled_data[train_size:]

# Create sequences for LSTM
def to_sequences(dataset, seq_size=1):
    x, y = [], []
    for i in range(len(dataset) - seq_size - 1):
        x.append(dataset[i:(i + seq_size), 0])
        y.append(dataset[i + seq_size, 0])
    return np.array(x), np.array(y)

seq_size = 3  # Number of time steps to look back
train_X, train_y = to_sequences(train, seq_size)
test_X, test_y = to_sequences(test, seq_size)

# Convert to PyTorch tensors
train_X = torch.tensor(train_X).float()
train_y = torch.tensor(train_y).view(-1, 1).float()
test_X = torch.tensor(test_X).float()
test_y = torch.tensor(test_y).view(-1, 1).float()

# Define the model
class NeuralNetwork(nn.Module):
    def __init__(self, input_dim, neurons, layers1, activation, dropout_rate):
        super(NeuralNetwork, self).__init__()
        self.layers = nn.ModuleList()
        self.layers.append(nn.Linear(input_dim, neurons))
        self.layers.append(getattr(nn, activation)())
        for _ in range(layers1):
            self.layers.append(nn.Linear(neurons, neurons))
            self.layers.append(getattr(nn, activation)())
            if dropout_rate > 0:
                self.layers.append(nn.Dropout(p=dropout_rate))
        self.layers.append(nn.Linear(neurons, 1))

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

# Set up the device (GPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Initialize the model, loss function, and optimizer
activation = 'ReLU'
neurons = 32
layers1 = 2
dropout_rate = 0.05
input_dim = train_X.shape[1]
learning_rate = 0.001

model = NeuralNetwork(input_dim, neurons, layers1, activation, dropout_rate).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Prepare the data loaders
batch_size = 64
train_dataset = TensorDataset(train_X, train_y)
test_dataset = TensorDataset(test_X, test_y)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Train the model and monitor power consumption
epochs = 60
train_losses, val_losses = [], []
min_val_loss = float('inf')

# Before starting training power monitoring
clear_log_file(training_power_log)

print("Starting training and logging power consumption...")
training_power_log = "training_power_log.csv"
monitor_process = monitor_power(training_power_log)

# ---- Add training time measurement ----
start_time = time.time()

for epoch in range(epochs):
    model.train()
    epoch_loss = 0
    for batch_X, batch_y in train_loader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for val_X, val_y in test_loader:
            val_X, val_y = val_X.to(device), val_y.to(device)
            val_outputs = model(val_X)
            val_loss += criterion(val_outputs, val_y).item()
    val_loss /= len(test_loader)
    train_losses.append(epoch_loss / len(train_loader))
    val_losses.append(val_loss)
    print(f"Epoch {epoch + 1}/{epochs}, Train Loss: {epoch_loss / len(train_loader):.6f}, Val Loss: {val_loss:.6f}")

    if val_loss < min_val_loss:
        min_val_loss = val_loss
        torch.save(model.state_dict(), 'best_model.pth')

# Stop the power monitoring for training
end_time = time.time()
monitor_process.terminate()
monitor_process.wait()

training_time = end_time - start_time
print(f"Training complete. Training time: {training_time:.2f} seconds.")
print("Calculating power consumption for training...")

# Calculate energy consumption for training
train_energy_joules, train_energy_kwh = calculate_energy_consumption(training_power_log)
print(f"Training Energy Consumption: {train_energy_joules:.2f} Joules ({train_energy_kwh:.6f} kWh)")


# Before starting inference power monitoring
clear_log_file(inference_power_log)


# Inference and monitor power consumption
inference_power_log = "inference_power_log.csv"
print("Starting inference and logging power consumption...")
monitor_process = monitor_power(inference_power_log)

# Add delay to ensure logging starts
time.sleep(1)

model.eval()
with torch.no_grad():
    for _ in range(1):  # Artificially increase inference workload
        predictions = model(test_X.to(device)).cpu().numpy()

# Add delay to ensure all logs are written
time.sleep(1)

# Stop the power monitoring for inference
monitor_process.terminate()
monitor_process.wait()
print("Inference complete. Calculating power consumption...")

# Calculate energy consumption for inference
inference_energy_joules, inference_energy_kwh = calculate_energy_consumption(inference_power_log)
print(f"Inference Energy Consumption: {inference_energy_joules:.2f} Joules ({inference_energy_kwh:.6f} kWh)")

# Rescale predictions and evaluate
predictions_unnorm = scaler.inverse_transform(predictions)
test_y_unnorm = scaler.inverse_transform(test_y.numpy())

mse = mean_squared_error(test_y_unnorm, predictions_unnorm)
rmse = np.sqrt(mse)
mae = mean_absolute_error(test_y_unnorm, predictions_unnorm)

print(f"Unnormalized MSE: {mse:.6f}")
print(f"Unnormalized RMSE: {rmse:.6f}")
print(f"Unnormalized MAE: {mae:.6f}")


<ipython-input-4-21e13144e22b>:50: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data = pd.read_csv('jordan_pv.csv', parse_dates=['date'], index_col='date')


Using device: cuda
Cleared existing log file: training_power_log.csv
Starting training and logging power consumption...
Epoch 1/60, Train Loss: 0.014148, Val Loss: 0.001350
Epoch 2/60, Train Loss: 0.004043, Val Loss: 0.001153
Epoch 3/60, Train Loss: 0.003589, Val Loss: 0.001363
Epoch 4/60, Train Loss: 0.003438, Val Loss: 0.001083
Epoch 5/60, Train Loss: 0.003368, Val Loss: 0.001042
Epoch 6/60, Train Loss: 0.003269, Val Loss: 0.001123
Epoch 7/60, Train Loss: 0.003242, Val Loss: 0.001035
Epoch 8/60, Train Loss: 0.003170, Val Loss: 0.000970
Epoch 9/60, Train Loss: 0.003160, Val Loss: 0.000874
Epoch 10/60, Train Loss: 0.003076, Val Loss: 0.000667
Epoch 11/60, Train Loss: 0.003076, Val Loss: 0.000970
Epoch 12/60, Train Loss: 0.002962, Val Loss: 0.000631
Epoch 13/60, Train Loss: 0.003008, Val Loss: 0.000714
Epoch 14/60, Train Loss: 0.003000, Val Loss: 0.000530
Epoch 15/60, Train Loss: 0.002979, Val Loss: 0.000579
Epoch 16/60, Train Loss: 0.002972, Val Loss: 0.000719
Epoch 17/60, Train Loss: 

In [ ]:
!nvidia-smi


Thu Jan  9 12:45:34 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
print(model)

NeuralNetwork(
  (layers): ModuleList(
    (0): Linear(in_features=3, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=32, bias=True)
    (3): ReLU()
    (4): Dropout(p=0.05, inplace=False)
    (5): Linear(in_features=32, out_features=32, bias=True)
    (6): ReLU()
    (7): Dropout(p=0.05, inplace=False)
    (8): Linear(in_features=32, out_features=1, bias=True)
  )
)


# CNN

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
from sklearn.preprocessing import MinMaxScaler
from pandas import read_csv
from sklearn.metrics import mean_absolute_error, mean_squared_error
import subprocess

# -----------------------------------------------------------------------
# 1) Functions for GPU Power Monitoring, as in the second script
# -----------------------------------------------------------------------
def monitor_power(output_file):
    """
    Run nvidia-smi in the background to log power usage (in W) every 1 second.
    """
    command = f"nvidia-smi --query-gpu=power.draw --format=csv -l 1 > {output_file}"
    process = subprocess.Popen(command, shell=True)
    return process

def calculate_energy_consumption(log_file):
    """
    Parse the CSV log with GPU power usage and calculate:
      - total energy in Joules
      - total energy in kWh
    """
    data = pd.read_csv(log_file, skiprows=1, names=["Power (W)"])

    # Clean the "Power (W)" column in case it contains the units
    data["Power (W)"] = data["Power (W)"].str.extract(r'(\d+\.\d+|\d+)').astype(float)

    # Each reading is 1 second apart, so total Joules is sum(power_in_watts) * 1 second
    energy_joules = data["Power (W)"].sum()

    # Convert Joules to kWh
    energy_kwh = energy_joules / (3600 * 1000)

    return energy_joules, energy_kwh
# Add this at the beginning of your script or before starting power monitoring
def clear_log_file(log_file):
    if os.path.exists(log_file):
        os.remove(log_file)
        print(f"Cleared existing log file: {log_file}")
# -----------------------------------------------------------------------
# 2) Data Loading and Preprocessing (same as your original first script)
# -----------------------------------------------------------------------
# Load and preprocess the dataset
data = read_csv('jordan_pv.csv', parse_dates=['date'], index_col='date')
#data = read_csv('Palestine-PV.csv', parse_dates=['date'], index_col='date')

# Convert index to datetime and sort
data.index = pd.to_datetime(data.index)
data = data.sort_index()

# Select the 6th column (index 5) for the target variable, remove NaN
data = data.iloc[:, 5].dropna().astype('float32')

# Convert to np.array and scale
data = np.array(data).reshape(-1, 1)
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

# Train/test split
train_size = int(len(scaled_data) * 0.80)
train, test = scaled_data[0:train_size, :], scaled_data[train_size:len(scaled_data), :]

def to_sequences(dataset, seq_size=1):
    x, y = [], []
    for i in range(len(dataset) - seq_size - 1):
        window = dataset[i:(i + seq_size), 0]
        x.append(window)
        y.append(dataset[i + seq_size, 0])
    return np.array(x), np.array(y)

seq_size = 3
train_X, train_y = to_sequences(train, seq_size)
test_X, test_y = to_sequences(test, seq_size)

# Convert to Torch tensors
train_X = torch.tensor(train_X).float()
train_y = torch.tensor(train_y).view(-1, 1).float()
test_X = torch.tensor(test_X).float()
test_y = torch.tensor(test_y).view(-1, 1).float()

# -----------------------------------------------------------------------
# 3) Define the Model
# -----------------------------------------------------------------------
class NeuralNetwork(nn.Module):
    def __init__(self, input_dim, neurons, layers1, activation, dropout_rate):
        super(NeuralNetwork, self).__init__()
        self.conv1 = nn.Conv1d(1, 32, kernel_size=1)
        self.conv2 = nn.Conv1d(32, 32, kernel_size=1)
        self.flatten = nn.Flatten()

        self.layers = nn.ModuleList()
        self.layers.append(nn.Linear(input_dim * 32, neurons))
        self.layers.append(getattr(nn, activation)())
        for _ in range(layers1):
            self.layers.append(nn.Linear(neurons, neurons))
            self.layers.append(getattr(nn, activation)())
            if dropout_rate > 0:
                self.layers.append(nn.Dropout(p=dropout_rate))

        self.output_layer = nn.Linear(neurons, 1)

    def forward(self, x):
        # x shape: (batch_size, 1, seq_length)
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.flatten(x)  # shape: (batch_size, seq_length * 32)
        for layer in self.layers:
            x = layer(x)
        x = self.output_layer(x)
        return x

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# -----------------------------------------------------------------------
# 4) Hyper-params and DataLoader
# -----------------------------------------------------------------------
activation = 'ReLU'
batch_size = 64
dropout_rate = 0.05
epochs = 60
layers1 = 2
learning_rate = 0.001
neurons = 32
input_dim = train_X.shape[1]

# Reshape for conv layers -> (batch, channels=1, seq_length)
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
test_X  = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))

train_dataset = TensorDataset(train_X, train_y)
test_dataset  = TensorDataset(test_X, test_y)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader  = DataLoader(test_dataset,  batch_size=batch_size, shuffle=False)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

model = NeuralNetwork(input_dim, neurons, layers1, activation, dropout_rate).to(device)
print(f"Trainable parameters: {count_parameters(model)}")

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# -----------------------------------------------------------------------
# 5) Monitor GPU power & Train
# -----------------------------------------------------------------------
# Before starting training power monitoring
clear_log_file(training_power_log)


print("Starting training and logging power consumption...")
train_power_log = "training_power_log.csv"
monitor_process_train = monitor_power(train_power_log)

start_time = time.time()
best_val_loss = float('inf')

for epoch in range(epochs):
    model.train()
    epoch_loss = 0.0
    for batch_X, batch_y in train_loader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    # Validation
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for val_X, val_y in test_loader:
            val_X, val_y = val_X.to(device), val_y.to(device)
            val_out = model(val_X)
            val_loss += criterion(val_out, val_y).item()

    val_loss /= len(test_loader)
    avg_train_loss = epoch_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{epochs}, Train Loss: {avg_train_loss:.6f}, Val Loss: {val_loss:.6f}")

    # Track best val loss
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), 'best_model.pth')

end_time = time.time()
monitor_process_train.terminate()
monitor_process_train.wait()

# Calculate training energy usage
print("Training complete. Calculating power consumption...")
train_energy_joules, train_energy_kwh = calculate_energy_consumption(train_power_log)
print(f"Training Time: {end_time - start_time:.2f} sec")
print(f"Training Energy Consumption: {train_energy_joules:.2f} Joules ({train_energy_kwh:.6f} kWh)")

# -----------------------------------------------------------------------
# 6) Monitor GPU Power & Inference
# -----------------------------------------------------------------------
clear_log_file(inference_power_log)

print("Starting inference and logging power consumption...")
inference_power_log = "inference_power_log.csv"
monitor_process_infer = monitor_power(inference_power_log)
time.sleep(1)  # give nvidia-smi a moment to start logging

model.eval()
with torch.no_grad():
    # Let’s artificially loop a few times to measure inference consumption
    for _ in range(1):
        preds = model(test_X.to(device))

# Stop logging for inference
time.sleep(1)
monitor_process_infer.terminate()
monitor_process_infer.wait()

inference_energy_joules, inference_energy_kwh = calculate_energy_consumption(inference_power_log)
print(f"Inference Energy Consumption: {inference_energy_joules:.2f} Joules ({inference_energy_kwh:.6f} kWh)")

# -----------------------------------------------------------------------
# 7) Evaluate Results (Unnormalized)
# -----------------------------------------------------------------------
predictions = preds.cpu().numpy()
predictions_unnorm = scaler.inverse_transform(predictions)
test_y_unnorm = scaler.inverse_transform(test_y.numpy())

mse_unnorm = mean_squared_error(test_y_unnorm, predictions_unnorm)
rmse_unnorm = np.sqrt(mse_unnorm)
mae_unnorm = mean_absolute_error(test_y_unnorm, predictions_unnorm)

print("Final Unnormalized Metrics (on last inference results):")
print(f" MSE:  {mse_unnorm:.6f}")
print(f" RMSE: {rmse_unnorm:.6f}")
print(f" MAE:  {mae_unnorm:.6f}")
print(f"Best Validation Loss (MSE): {best_val_loss:.6f}")


<ipython-input-6-99d867dcf0f5>:52: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data = read_csv('jordan_pv.csv', parse_dates=['date'], index_col='date')


Using device: cuda
Trainable parameters: 6369
Cleared existing log file: training_power_log.csv
Starting training and logging power consumption...
Epoch 1/60, Train Loss: 0.013092, Val Loss: 0.001701
Epoch 2/60, Train Loss: 0.004034, Val Loss: 0.000992
Epoch 3/60, Train Loss: 0.003650, Val Loss: 0.003101
Epoch 4/60, Train Loss: 0.003422, Val Loss: 0.000990
Epoch 5/60, Train Loss: 0.003404, Val Loss: 0.001139
Epoch 6/60, Train Loss: 0.003214, Val Loss: 0.000734
Epoch 7/60, Train Loss: 0.003211, Val Loss: 0.000893
Epoch 8/60, Train Loss: 0.003165, Val Loss: 0.000614
Epoch 9/60, Train Loss: 0.003171, Val Loss: 0.000891
Epoch 10/60, Train Loss: 0.003081, Val Loss: 0.000648
Epoch 11/60, Train Loss: 0.003055, Val Loss: 0.000745
Epoch 12/60, Train Loss: 0.003105, Val Loss: 0.001131
Epoch 13/60, Train Loss: 0.003059, Val Loss: 0.000685
Epoch 14/60, Train Loss: 0.003022, Val Loss: 0.001005
Epoch 15/60, Train Loss: 0.003019, Val Loss: 0.000635
Epoch 16/60, Train Loss: 0.002942, Val Loss: 0.00140

In [ ]:
print(model)

NeuralNetwork(
  (conv1): Conv1d(1, 32, kernel_size=(1,), stride=(1,))
  (conv2): Conv1d(32, 32, kernel_size=(1,), stride=(1,))
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (layers): ModuleList(
    (0): Linear(in_features=576, out_features=100, bias=True)
    (1): ReLU()
    (2): Linear(in_features=100, out_features=100, bias=True)
    (3): ReLU()
    (4): Linear(in_features=100, out_features=100, bias=True)
    (5): ReLU()
  )
  (output_layer): Linear(in_features=100, out_features=1, bias=True)
)


# LSTM

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import pandas as pd
import time
import subprocess
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from pandas import read_csv
import os
# ---------------------------------------------------------
# 1) GPU power monitoring functions
# ---------------------------------------------------------
def monitor_power(output_file):
    """
    Launch nvidia-smi in the background, logging instantaneous GPU power usage (W) every 1 second.
    """
    command = f"nvidia-smi --query-gpu=power.draw --format=csv -l 1 > {output_file}"
    process = subprocess.Popen(command, shell=True)
    return process

def calculate_energy_consumption(log_file):
    """
    Read the CSV log with GPU power (in W) every second.
    Sum them up to get total Joules. Then convert Joules -> kWh.
    """
    data = pd.read_csv(log_file, skiprows=1, names=["Power (W)"])
    # Clean up any trailing " W" or similar
    data["Power (W)"] = data["Power (W)"].str.extract(r'(\d+\.\d+|\d+)').astype(float)
    # Energy(J) = sum of (Power_in_W * 1 second)
    energy_joules = data["Power (W)"].sum()
    # Convert to kWh
    energy_kwh = energy_joules / (3600 * 1000)
    return energy_joules, energy_kwh

# Add this at the beginning of your script or before starting power monitoring
def clear_log_file(log_file):
    if os.path.exists(log_file):
        os.remove(log_file)
        print(f"Cleared existing log file: {log_file}")

# ---------------------------------------------------------
# 2) Load & preprocess data
# ---------------------------------------------------------
data = read_csv('jordan_pv.csv', parse_dates=['date'], index_col='date')
#data = read_csv('Palestine-PV.csv', parse_dates=['date'], index_col='date')

data.index = pd.to_datetime(data.index)
data = data.sort_index()

# Select column 6 (index 5), drop NaN
data = data.iloc[:, 5].dropna().astype('float32')

# Convert to numpy and scale
values = np.array(data).reshape(-1, 1)
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(values)

# Train/test split
train_size = int(len(scaled_data) * 0.8)
train_data = scaled_data[:train_size]
test_data = scaled_data[train_size:]

def to_sequences(dataset, seq_len=1):
    x, y = [], []
    for i in range(len(dataset) - seq_len - 1):
        x.append(dataset[i:(i + seq_len), 0])
        y.append(dataset[i + seq_len, 0])
    return np.array(x), np.array(y)

seq_size = 3
train_X, train_y = to_sequences(train_data, seq_size)
test_X, test_y = to_sequences(test_data, seq_size)

# Convert to Torch tensor
train_X = torch.tensor(train_X).float()
train_y = torch.tensor(train_y).view(-1,1).float()
test_X  = torch.tensor(test_X).float()
test_y  = torch.tensor(test_y).view(-1,1).float()

# ---------------------------------------------------------
# 3) Define the LSTM-based model
# ---------------------------------------------------------
class LSTMNetwork(nn.Module):
    def __init__(self, input_dim, neurons, layers1, activation, dropout_rate):
        super(LSTMNetwork, self).__init__()
        self.lstm = nn.LSTM(input_dim, neurons, layers1, batch_first=True)
        self.layers = nn.ModuleList()
        self.layers.append(nn.Linear(neurons, neurons))
        self.layers.append(getattr(nn, activation)())
        self.layers.append(nn.Dropout(p=dropout_rate))  # Dropout after first dense layer
        for _ in range(layers1 - 1):
            self.layers.append(nn.Linear(neurons, neurons))
            self.layers.append(getattr(nn, activation)())
            if dropout_rate > 0:
                self.layers.append(nn.Dropout(p=dropout_rate))  # Dropout after each dense layer
        self.output_layer = nn.Linear(neurons, 1)

    def forward(self, x):
        x, _ = self.lstm(x)
        x = x[:, -1, :]  # get the last output of the sequence
        for layer in self.layers:
            x = layer(x)
        x = self.output_layer(x)
        return x

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# ---------------------------------------------------------
# 4) Hyperparams, DataLoaders, & Setup
# ---------------------------------------------------------
activation='ReLU'
neurons=32
layers1=2
dropout_rate=0.05
epochs=60
learning_rate=0.001
batch_size=64

input_dim = train_X.shape[1]  # seq_size
# Reshape for conv -> shape: (N, channels=1, seq_len)
train_X = train_X.reshape(train_X.shape[0], 1, train_X.shape[1])
test_X  = test_X.reshape(test_X.shape[0], 1, test_X.shape[1])

train_dataset = TensorDataset(train_X, train_y)
test_dataset  = TensorDataset(test_X, test_y)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

model = LSTMNetwork(input_dim, neurons, layers1, activation, dropout_rate).to(device)
nparams = count_parameters(model)
print(f"Trainable parameters: {nparams}")

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# ---------------------------------------------------------
# 5) TRAINING - Monitor GPU power
# ---------------------------------------------------------
# Before starting training power monitoring
clear_log_file(training_power_log)
train_power_log = "training_power_log.csv"
print("Starting training and logging power consumption...")
monitor_proc = monitor_power(train_power_log)
start_time = time.time()

best_val_loss = float('inf')
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for batchX, batchY in train_loader:
        batchX, batchY = batchX.to(device), batchY.to(device)
        optimizer.zero_grad()
        out = model(batchX)
        loss = criterion(out, batchY)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    train_loss_epoch = running_loss / len(train_loader)

    # Validation
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for valX, valY in test_loader:
            valX, valY = valX.to(device), valY.to(device)
            predY = model(valX)
            val_loss += criterion(predY, valY).item()
    val_loss /= len(test_loader)

    print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_loss_epoch:.6f}, Val Loss: {val_loss:.6f}")
    # Track best val
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), "best_model.pth")

end_time = time.time()
monitor_proc.terminate()
monitor_proc.wait()

train_energy_joules, train_energy_kwh = calculate_energy_consumption(train_power_log)
train_time = end_time - start_time
print("Training completed!")
print(f"Training Time: {train_time:.2f} sec")
print(f"Training Energy Consumption: {train_energy_joules:.2f} Joules ({train_energy_kwh:.6f} kWh)")

# ---------------------------------------------------------
# 6) INFERENCE - Monitor GPU power
# ---------------------------------------------------------
# Before starting inference power monitoring
clear_log_file(inference_power_log)

inference_power_log = "inference_power_log.csv"
print("Starting inference and logging power consumption...")
monitor_proc_infer = monitor_power(inference_power_log)
time.sleep(1)  # give nvidia-smi time to start logging

model.eval()
with torch.no_grad():
    # artificially enlarge the inference workload
    for _ in range(1):
        predictions = model(test_X.to(device))

time.sleep(1)  # ensure last logs are written
monitor_proc_infer.terminate()
monitor_proc_infer.wait()

inference_energy_joules, inference_energy_kwh = calculate_energy_consumption(inference_power_log)
print("Inference Completed!")
print(f"Inference Energy Consumption: {inference_energy_joules:.2f} Joules ({inference_energy_kwh:.6f} kWh)")

# ---------------------------------------------------------
# 7) Print out all final metrics, exactly as in the original script
# ---------------------------------------------------------
predictions_np = predictions.cpu().numpy()
test_y_np = test_y.cpu().numpy()

# Inverse transform
pred_unnorm = scaler.inverse_transform(predictions_np)
test_y_unnorm = scaler.inverse_transform(test_y_np)

# Compute MSE, RMSE, MAE on unnormalized data
mse_unnorm = mean_squared_error(test_y_unnorm, pred_unnorm)
rmse_unnorm = np.sqrt(mse_unnorm)
mae_unnorm = mean_absolute_error(test_y_unnorm, pred_unnorm)

print("\nFinal Unnormalized Metrics (on last inference results):")
print(f" MSE:  {mse_unnorm:.6f}")
print(f" RMSE: {rmse_unnorm:.6f}")
print(f" MAE:  {mae_unnorm:.6f}")
print(f"Best Validation Loss (MSE): {best_val_loss:.6f}")


<ipython-input-8-653145e9ab7b>:47: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data = read_csv('jordan_pv.csv', parse_dates=['date'], index_col='date')


Using device: cuda
Trainable parameters: 15329
Cleared existing log file: training_power_log.csv
Starting training and logging power consumption...
Epoch 1/60, Train Loss: 0.037486, Val Loss: 0.001621
Epoch 2/60, Train Loss: 0.004545, Val Loss: 0.001941
Epoch 3/60, Train Loss: 0.004216, Val Loss: 0.001443
Epoch 4/60, Train Loss: 0.003885, Val Loss: 0.001006
Epoch 5/60, Train Loss: 0.003747, Val Loss: 0.000736
Epoch 6/60, Train Loss: 0.003550, Val Loss: 0.000998
Epoch 7/60, Train Loss: 0.003496, Val Loss: 0.000765
Epoch 8/60, Train Loss: 0.003333, Val Loss: 0.000742
Epoch 9/60, Train Loss: 0.003311, Val Loss: 0.000663
Epoch 10/60, Train Loss: 0.003212, Val Loss: 0.000675
Epoch 11/60, Train Loss: 0.003246, Val Loss: 0.000983
Epoch 12/60, Train Loss: 0.003178, Val Loss: 0.001312
Epoch 13/60, Train Loss: 0.003205, Val Loss: 0.000820
Epoch 14/60, Train Loss: 0.003098, Val Loss: 0.000999
Epoch 15/60, Train Loss: 0.003135, Val Loss: 0.000628
Epoch 16/60, Train Loss: 0.003066, Val Loss: 0.0017

In [ ]:
print(model)

LSTMNetwork(
  (lstm): LSTM(3, 32, num_layers=2, batch_first=True)
  (layers): ModuleList(
    (0): Linear(in_features=32, out_features=32, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.05, inplace=False)
    (3): Linear(in_features=32, out_features=32, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.05, inplace=False)
  )
  (output_layer): Linear(in_features=32, out_features=1, bias=True)
)


In [ ]:
# MLP Mutivarite

In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
import subprocess
import time

# Function to monitor power usage with nvidia-smi
def monitor_power(output_file):
    # Run nvidia-smi in the background to log power usage
    command = f"nvidia-smi --query-gpu=power.draw --format=csv -l 1 > {output_file}"
    process = subprocess.Popen(command, shell=True)
    return process

# Function to calculate energy consumption from the log
def calculate_energy_consumption(log_file):
    # Load the power log file
    data = pd.read_csv(log_file, skiprows=1, names=["Power (W)"])

    # Debug: Check log content
    print(f"Log file content:\n{data.head()}")

    # Clean the "Power (W)" column to remove non-numeric characters (e.g., " W")
    data["Power (W)"] = data["Power (W)"].str.extract(r'(\d+\.\d+|\d+)').astype(float)

    # Debug: Check cleaned data
    print(f"Cleaned log data:\n{data.head()}")

    # Calculate total energy in Joules (assuming logs are every 1 second)
    energy_joules = data["Power (W)"].sum()  # Energy = Power × Time (1 second per log)

    # Convert total energy to kWh
    energy_kwh = energy_joules / (3600 * 1000)

    return energy_joules, energy_kwh
# Add this at the beginning of your script or before starting power monitoring
def clear_log_file(log_file):
    if os.path.exists(log_file):
        os.remove(log_file)
        print(f"Cleared existing log file: {log_file}")

# Load and preprocess the dataset
#dataset = pd.read_csv('Palestine-PV.csv', parse_dates=['date'], index_col='date')
dataset = pd.read_csv('jordan_pv.csv',  index_col='date')
dataset.index = pd.to_datetime(dataset.index)
dataset = dataset.sort_index()
dataset = dataset.dropna()
dataset = dataset.astype(np.float64)

def series_to_supervised(data, target_col_name, n_in=1, n_out=1, dropnan=True):
    n_vars = data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    df = df[[col for col in df if col != target_col_name] + [target_col_name]]
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars-1)]
            names.append(target_col_name + '(t)')
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    agg = agg[[col for col in agg if target_col_name in col or '(t)' not in col]]
    if dropnan:
        agg.dropna(inplace=True)
    return agg

values = dataset.values
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)
df_scaled = pd.DataFrame(scaled, columns=dataset.columns)
target_scaler = MinMaxScaler(feature_range=(0, 1))
target_scaled = target_scaler.fit_transform(dataset[['Solar Radiation(GHI)']])
reframed = series_to_supervised(df_scaled, 'Solar Radiation(GHI)', 3, 1)
values = reframed.values
n_train_hours = 876 * 24
train = values[:n_train_hours, :]
test = values[n_train_hours:, :]
train_X, train_y = train[:, :-1], train[:, -1]
test_X, test_y = test[:, :-1], test[:, -1]
train_X, train_y = torch.Tensor(train_X), torch.Tensor(train_y).view(-1, 1)
test_X, test_y = torch.Tensor(test_X), torch.Tensor(test_y).view(-1, 1)


# Define the model
class NeuralNetwork(nn.Module):
    def __init__(self, input_dim, neurons, layers1, activation, dropout_rate):
        super(NeuralNetwork, self).__init__()
        self.layers = nn.ModuleList()
        self.layers.append(nn.Linear(input_dim, neurons))
        self.layers.append(getattr(nn, activation)())
        for _ in range(layers1):
            self.layers.append(nn.Linear(neurons, neurons))
            self.layers.append(getattr(nn, activation)())
            if dropout_rate > 0:
                self.layers.append(nn.Dropout(p=dropout_rate))
        self.layers.append(nn.Linear(neurons, 1))

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

# Set up the device (GPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Initialize the model, loss function, and optimizer
activation = 'ReLU'
neurons = 32
layers1 = 2
dropout_rate = 0.05
input_dim = train_X.shape[1]
learning_rate = 0.001

model = NeuralNetwork(input_dim, neurons, layers1, activation, dropout_rate).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Prepare the data loaders
batch_size = 64
train_dataset = TensorDataset(train_X, train_y)
test_dataset = TensorDataset(test_X, test_y)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Train the model and monitor power consumption
epochs = 60
train_losses, val_losses = [], []
min_val_loss = float('inf')

# Before starting training power monitoring
clear_log_file(training_power_log)

print("Starting training and logging power consumption...")
training_power_log = "training_power_log.csv"
monitor_process = monitor_power(training_power_log)

# ---- Add training time measurement ----
start_time = time.time()

for epoch in range(epochs):
    model.train()
    epoch_loss = 0
    for batch_X, batch_y in train_loader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for val_X, val_y in test_loader:
            val_X, val_y = val_X.to(device), val_y.to(device)
            val_outputs = model(val_X)
            val_loss += criterion(val_outputs, val_y).item()
    val_loss /= len(test_loader)
    train_losses.append(epoch_loss / len(train_loader))
    val_losses.append(val_loss)
    print(f"Epoch {epoch + 1}/{epochs}, Train Loss: {epoch_loss / len(train_loader):.6f}, Val Loss: {val_loss:.6f}")

    if val_loss < min_val_loss:
        min_val_loss = val_loss
        torch.save(model.state_dict(), 'best_model.pth')

# Stop the power monitoring for training
end_time = time.time()
monitor_process.terminate()
monitor_process.wait()

training_time = end_time - start_time
print(f"Training complete. Training time: {training_time:.2f} seconds.")
print("Calculating power consumption for training...")

# Calculate energy consumption for training
train_energy_joules, train_energy_kwh = calculate_energy_consumption(training_power_log)
print(f"Training Energy Consumption: {train_energy_joules:.2f} Joules ({train_energy_kwh:.6f} kWh)")

# Before starting inference power monitoring
clear_log_file(inference_power_log)

# Inference and monitor power consumption
inference_power_log = "inference_power_log.csv"
print("Starting inference and logging power consumption...")
monitor_process = monitor_power(inference_power_log)

# Add delay to ensure logging starts
time.sleep(1)

model.eval()
with torch.no_grad():
    for _ in range(1):  # Artificially increase inference workload
        predictions = model(test_X.to(device)).cpu().numpy()

# Add delay to ensure all logs are written
time.sleep(1)

# Stop the power monitoring for inference
monitor_process.terminate()
monitor_process.wait()
print("Inference complete. Calculating power consumption...")

# Calculate energy consumption for inference
inference_energy_joules, inference_energy_kwh = calculate_energy_consumption(inference_power_log)
print(f"Inference Energy Consumption: {inference_energy_joules:.2f} Joules ({inference_energy_kwh:.6f} kWh)")

# Rescale predictions and evaluate
predictions_unnorm = scaler.inverse_transform(predictions)
test_y_unnorm = scaler.inverse_transform(test_y.numpy())

mse = mean_squared_error(test_y_unnorm, predictions_unnorm)
rmse = np.sqrt(mse)
mae = mean_absolute_error(test_y_unnorm, predictions_unnorm)

print(f"Unnormalized MSE: {mse:.6f}")
print(f"Unnormalized RMSE: {rmse:.6f}")
print(f"Unnormalized MAE: {mae:.6f}")


<ipython-input-10-5d0b188e2c77>:49: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dataset.index = pd.to_datetime(dataset.index)


Using device: cuda
Cleared existing log file: training_power_log.csv
Starting training and logging power consumption...
Epoch 1/60, Train Loss: 0.020542, Val Loss: 0.001442
Epoch 2/60, Train Loss: 0.004146, Val Loss: 0.000958
Epoch 3/60, Train Loss: 0.003682, Val Loss: 0.000825
Epoch 4/60, Train Loss: 0.003380, Val Loss: 0.000682
Epoch 5/60, Train Loss: 0.003201, Val Loss: 0.000817
Epoch 6/60, Train Loss: 0.003074, Val Loss: 0.000633
Epoch 7/60, Train Loss: 0.003015, Val Loss: 0.000820
Epoch 8/60, Train Loss: 0.002894, Val Loss: 0.000471
Epoch 9/60, Train Loss: 0.002968, Val Loss: 0.000624
Epoch 10/60, Train Loss: 0.002971, Val Loss: 0.000582
Epoch 11/60, Train Loss: 0.002847, Val Loss: 0.000529
Epoch 12/60, Train Loss: 0.002721, Val Loss: 0.000433
Epoch 13/60, Train Loss: 0.002746, Val Loss: 0.000783
Epoch 14/60, Train Loss: 0.002677, Val Loss: 0.000470
Epoch 15/60, Train Loss: 0.002690, Val Loss: 0.000484
Epoch 16/60, Train Loss: 0.002650, Val Loss: 0.000994
Epoch 17/60, Train Loss: 

ValueError: non-broadcastable output operand with shape (5253,1) doesn't match the broadcast shape (5253,6)

In [ ]:
# CNN Multivariate

In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
from sklearn.preprocessing import MinMaxScaler
from pandas import read_csv
from sklearn.metrics import mean_absolute_error, mean_squared_error
import subprocess

# -----------------------------------------------------------------------
# 1) Functions for GPU Power Monitoring, as in the second script
# -----------------------------------------------------------------------
def monitor_power(output_file):
    """
    Run nvidia-smi in the background to log power usage (in W) every 1 second.
    """
    command = f"nvidia-smi --query-gpu=power.draw --format=csv -l 1 > {output_file}"
    process = subprocess.Popen(command, shell=True)
    return process

def calculate_energy_consumption(log_file):
    """
    Parse the CSV log with GPU power usage and calculate:
      - total energy in Joules
      - total energy in kWh
    """
    data = pd.read_csv(log_file, skiprows=1, names=["Power (W)"])

    # Clean the "Power (W)" column in case it contains the units
    data["Power (W)"] = data["Power (W)"].str.extract(r'(\d+\.\d+|\d+)').astype(float)

    # Each reading is 1 second apart, so total Joules is sum(power_in_watts) * 1 second
    energy_joules = data["Power (W)"].sum()

    # Convert Joules to kWh
    energy_kwh = energy_joules / (3600 * 1000)

    return energy_joules, energy_kwh

# Add this at the beginning of your script or before starting power monitoring
def clear_log_file(log_file):
    if os.path.exists(log_file):
        os.remove(log_file)
        print(f"Cleared existing log file: {log_file}")

# Load and preprocess the dataset
#dataset = pd.read_csv('Palestine-PV.csv', parse_dates=['date'], index_col='date')
dataset = pd.read_csv('jordan_pv.csv',  index_col='date')
dataset.index = pd.to_datetime(dataset.index)
dataset = dataset.sort_index()
dataset = dataset.dropna()
dataset = dataset.astype(np.float64)

def series_to_supervised(data, target_col_name, n_in=1, n_out=1, dropnan=True):
    n_vars = data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    df = df[[col for col in df if col != target_col_name] + [target_col_name]]
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars-1)]
            names.append(target_col_name + '(t)')
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    agg = agg[[col for col in agg if target_col_name in col or '(t)' not in col]]
    if dropnan:
        agg.dropna(inplace=True)
    return agg

values = dataset.values
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)
df_scaled = pd.DataFrame(scaled, columns=dataset.columns)
target_scaler = MinMaxScaler(feature_range=(0, 1))
target_scaled = target_scaler.fit_transform(dataset[['Solar Radiation(GHI)']])
reframed = series_to_supervised(df_scaled, 'Solar Radiation(GHI)', 3, 1)
values = reframed.values
n_train_hours = 876 * 24
train = values[:n_train_hours, :]
test = values[n_train_hours:, :]
train_X, train_y = train[:, :-1], train[:, -1]
test_X, test_y = test[:, :-1], test[:, -1]
train_X, train_y = torch.Tensor(train_X), torch.Tensor(train_y).view(-1, 1)
test_X, test_y = torch.Tensor(test_X), torch.Tensor(test_y).view(-1, 1)
# -----------------------------------------------------------------------
# 3) Define the Model
# -----------------------------------------------------------------------
class NeuralNetwork(nn.Module):
    def __init__(self, input_dim, neurons, layers1, activation, dropout_rate):
        super(NeuralNetwork, self).__init__()
        self.conv1 = nn.Conv1d(1, 32, kernel_size=1)
        self.conv2 = nn.Conv1d(32, 32, kernel_size=1)
        self.flatten = nn.Flatten()

        self.layers = nn.ModuleList()
        self.layers.append(nn.Linear(input_dim * 32, neurons))
        self.layers.append(getattr(nn, activation)())
        for _ in range(layers1):
            self.layers.append(nn.Linear(neurons, neurons))
            self.layers.append(getattr(nn, activation)())
            if dropout_rate > 0:
                self.layers.append(nn.Dropout(p=dropout_rate))

        self.output_layer = nn.Linear(neurons, 1)

    def forward(self, x):
        # x shape: (batch_size, 1, seq_length)
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.flatten(x)  # shape: (batch_size, seq_length * 32)
        for layer in self.layers:
            x = layer(x)
        x = self.output_layer(x)
        return x

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# -----------------------------------------------------------------------
# 4) Hyper-params and DataLoader
# -----------------------------------------------------------------------
activation = 'ReLU'
batch_size = 64
dropout_rate = 0.05
epochs = 60
layers1 = 2
learning_rate = 0.001
neurons = 32
input_dim = train_X.shape[1]

# Reshape for conv layers -> (batch, channels=1, seq_length)
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
test_X  = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))

train_dataset = TensorDataset(train_X, train_y)
test_dataset  = TensorDataset(test_X, test_y)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader  = DataLoader(test_dataset,  batch_size=batch_size, shuffle=False)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

model = NeuralNetwork(input_dim, neurons, layers1, activation, dropout_rate).to(device)
print(f"Trainable parameters: {count_parameters(model)}")

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# -----------------------------------------------------------------------
# 5) Monitor GPU power & Train
# -----------------------------------------------------------------------

# Before starting training power monitoring
clear_log_file(training_power_log)

print("Starting training and logging power consumption...")
train_power_log = "training_power_log.csv"
monitor_process_train = monitor_power(train_power_log)

start_time = time.time()
best_val_loss = float('inf')

for epoch in range(epochs):
    model.train()
    epoch_loss = 0.0
    for batch_X, batch_y in train_loader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    # Validation
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for val_X, val_y in test_loader:
            val_X, val_y = val_X.to(device), val_y.to(device)
            val_out = model(val_X)
            val_loss += criterion(val_out, val_y).item()

    val_loss /= len(test_loader)
    avg_train_loss = epoch_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{epochs}, Train Loss: {avg_train_loss:.6f}, Val Loss: {val_loss:.6f}")

    # Track best val loss
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), 'best_model.pth')

end_time = time.time()
monitor_process_train.terminate()
monitor_process_train.wait()

# Calculate training energy usage
print("Training complete. Calculating power consumption...")
train_energy_joules, train_energy_kwh = calculate_energy_consumption(train_power_log)
print(f"Training Time: {end_time - start_time:.2f} sec")
print(f"Training Energy Consumption: {train_energy_joules:.2f} Joules ({train_energy_kwh:.6f} kWh)")

# -----------------------------------------------------------------------
# 6) Monitor GPU Power & Inference
# -----------------------------------------------------------------------
# Before starting inference power monitoring
clear_log_file(inference_power_log)

print("Starting inference and logging power consumption...")
inference_power_log = "inference_power_log.csv"
monitor_process_infer = monitor_power(inference_power_log)
time.sleep(1)  # give nvidia-smi a moment to start logging

model.eval()
with torch.no_grad():
    # Let’s artificially loop a few times to measure inference consumption
    for _ in range(1):
        preds = model(test_X.to(device))

# Stop logging for inference
time.sleep(1)
monitor_process_infer.terminate()
monitor_process_infer.wait()

inference_energy_joules, inference_energy_kwh = calculate_energy_consumption(inference_power_log)
print(f"Inference Energy Consumption: {inference_energy_joules:.2f} Joules ({inference_energy_kwh:.6f} kWh)")

# -----------------------------------------------------------------------
# 7) Evaluate Results (Unnormalized)
# -----------------------------------------------------------------------
predictions = preds.cpu().numpy()
predictions_unnorm = scaler.inverse_transform(predictions)
test_y_unnorm = scaler.inverse_transform(test_y.numpy())

mse_unnorm = mean_squared_error(test_y_unnorm, predictions_unnorm)
rmse_unnorm = np.sqrt(mse_unnorm)
mae_unnorm = mean_absolute_error(test_y_unnorm, predictions_unnorm)

print("Final Unnormalized Metrics (on last inference results):")
print(f" MSE:  {mse_unnorm:.6f}")
print(f" RMSE: {rmse_unnorm:.6f}")
print(f" MAE:  {mae_unnorm:.6f}")
print(f"Best Validation Loss (MSE): {best_val_loss:.6f}")


<ipython-input-12-127531157a1b>:53: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dataset.index = pd.to_datetime(dataset.index)


Using device: cuda
Trainable parameters: 21729
Cleared existing log file: training_power_log.csv
Starting training and logging power consumption...
Epoch 1/60, Train Loss: 0.014300, Val Loss: 0.000739
Epoch 2/60, Train Loss: 0.003645, Val Loss: 0.000877
Epoch 3/60, Train Loss: 0.003291, Val Loss: 0.001006
Epoch 4/60, Train Loss: 0.003042, Val Loss: 0.000474
Epoch 5/60, Train Loss: 0.002969, Val Loss: 0.001270
Epoch 6/60, Train Loss: 0.002891, Val Loss: 0.000870
Epoch 7/60, Train Loss: 0.002774, Val Loss: 0.000939
Epoch 8/60, Train Loss: 0.002668, Val Loss: 0.000704
Epoch 9/60, Train Loss: 0.002698, Val Loss: 0.000638
Epoch 10/60, Train Loss: 0.002612, Val Loss: 0.000367
Epoch 11/60, Train Loss: 0.002589, Val Loss: 0.001295
Epoch 12/60, Train Loss: 0.002605, Val Loss: 0.002057
Epoch 13/60, Train Loss: 0.002614, Val Loss: 0.000700
Epoch 14/60, Train Loss: 0.002559, Val Loss: 0.001746
Epoch 15/60, Train Loss: 0.002478, Val Loss: 0.000549
Epoch 16/60, Train Loss: 0.002437, Val Loss: 0.0005

ValueError: non-broadcastable output operand with shape (5253,1) doesn't match the broadcast shape (5253,6)

In [ ]:
# LSTM Multivarite

In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import pandas as pd
import time
import subprocess
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from pandas import read_csv

# ---------------------------------------------------------
# 1) GPU power monitoring functions
# ---------------------------------------------------------
def monitor_power(output_file):
    """
    Launch nvidia-smi in the background, logging instantaneous GPU power usage (W) every 1 second.
    """
    command = f"nvidia-smi --query-gpu=power.draw --format=csv -l 1 > {output_file}"
    process = subprocess.Popen(command, shell=True)
    return process

def calculate_energy_consumption(log_file):
    """
    Read the CSV log with GPU power (in W) every second.
    Sum them up to get total Joules. Then convert Joules -> kWh.
    """
    data = pd.read_csv(log_file, skiprows=1, names=["Power (W)"])
    # Clean up any trailing " W" or similar
    data["Power (W)"] = data["Power (W)"].str.extract(r'(\d+\.\d+|\d+)').astype(float)
    # Energy(J) = sum of (Power_in_W * 1 second)
    energy_joules = data["Power (W)"].sum()
    # Convert to kWh
    energy_kwh = energy_joules / (3600 * 1000)
    return energy_joules, energy_kwh
def clear_log_file(log_file):
    if os.path.exists(log_file):
        os.remove(log_file)
        print(f"Cleared existing log file: {log_file}")
# ---------------------------------------------------------
# 2) Load & preprocess data
# Load and preprocess the dataset
#dataset = pd.read_csv('Palestine-PV.csv', parse_dates=['date'], index_col='date')
dataset = pd.read_csv('jordan_pv.csv',  index_col='date')
dataset.index = pd.to_datetime(dataset.index)
dataset = dataset.sort_index()
dataset = dataset.dropna()
dataset = dataset.astype(np.float64)

def series_to_supervised(data, target_col_name, n_in=1, n_out=1, dropnan=True):
    n_vars = data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    df = df[[col for col in df if col != target_col_name] + [target_col_name]]
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars-1)]
            names.append(target_col_name + '(t)')
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    agg = agg[[col for col in agg if target_col_name in col or '(t)' not in col]]
    if dropnan:
        agg.dropna(inplace=True)
    return agg

values = dataset.values
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)
df_scaled = pd.DataFrame(scaled, columns=dataset.columns)
target_scaler = MinMaxScaler(feature_range=(0, 1))
target_scaled = target_scaler.fit_transform(dataset[['Solar Radiation(GHI)']])
reframed = series_to_supervised(df_scaled, 'Solar Radiation(GHI)', 3, 1)
values = reframed.values
n_train_hours = 876 * 24
train = values[:n_train_hours, :]
test = values[n_train_hours:, :]
train_X, train_y = train[:, :-1], train[:, -1]
test_X, test_y = test[:, :-1], test[:, -1]
train_X, train_y = torch.Tensor(train_X), torch.Tensor(train_y).view(-1, 1)
test_X, test_y = torch.Tensor(test_X), torch.Tensor(test_y).view(-1, 1)

# ---------------------------------------------------------
# 3) Define the Conv1D-based model
# ---------------------------------------------------------
class LSTMNetwork(nn.Module):
    def __init__(self, input_dim, neurons, layers1, activation, dropout_rate):
        super(LSTMNetwork, self).__init__()
        self.lstm = nn.LSTM(input_dim, neurons, layers1, batch_first=True)
        self.layers = nn.ModuleList()
        self.layers.append(nn.Linear(neurons, neurons))
        self.layers.append(getattr(nn, activation)())
        self.layers.append(nn.Dropout(p=dropout_rate))  # Dropout after first dense layer
        for _ in range(layers1 - 1):
            self.layers.append(nn.Linear(neurons, neurons))
            self.layers.append(getattr(nn, activation)())
            if dropout_rate > 0:
                self.layers.append(nn.Dropout(p=dropout_rate))  # Dropout after each dense layer
        self.output_layer = nn.Linear(neurons, 1)

    def forward(self, x):
        x, _ = self.lstm(x)
        x = x[:, -1, :]  # get the last output of the sequence
        for layer in self.layers:
            x = layer(x)
        x = self.output_layer(x)
        return x

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# ---------------------------------------------------------
# 4) Hyperparams, DataLoaders, & Setup
# ---------------------------------------------------------
activation='ReLU'
neurons=32
layers1=2
dropout_rate=0.05
epochs=60
learning_rate=0.001
batch_size=64

input_dim = train_X.shape[1]  # seq_size
# Reshape for conv -> shape: (N, channels=1, seq_len)
train_X = train_X.reshape(train_X.shape[0], 1, train_X.shape[1])
test_X  = test_X.reshape(test_X.shape[0], 1, test_X.shape[1])

train_dataset = TensorDataset(train_X, train_y)
test_dataset  = TensorDataset(test_X, test_y)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

model = LSTMNetwork(input_dim, neurons, layers1, activation, dropout_rate).to(device)
nparams = count_parameters(model)
print(f"Trainable parameters: {nparams}")

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# ---------------------------------------------------------
# 5) TRAINING - Monitor GPU power
# ---------------------------------------------------------
# Before starting training power monitoring
clear_log_file(training_power_log)

train_power_log = "training_power_log.csv"
print("Starting training and logging power consumption...")
monitor_proc = monitor_power(train_power_log)
start_time = time.time()

best_val_loss = float('inf')
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for batchX, batchY in train_loader:
        batchX, batchY = batchX.to(device), batchY.to(device)
        optimizer.zero_grad()
        out = model(batchX)
        loss = criterion(out, batchY)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    train_loss_epoch = running_loss / len(train_loader)

    # Validation
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for valX, valY in test_loader:
            valX, valY = valX.to(device), valY.to(device)
            predY = model(valX)
            val_loss += criterion(predY, valY).item()
    val_loss /= len(test_loader)

    print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_loss_epoch:.6f}, Val Loss: {val_loss:.6f}")
    # Track best val
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), "best_model.pth")

end_time = time.time()
monitor_proc.terminate()
monitor_proc.wait()

train_energy_joules, train_energy_kwh = calculate_energy_consumption(train_power_log)
train_time = end_time - start_time
print("Training completed!")
print(f"Training Time: {train_time:.2f} sec")
print(f"Training Energy Consumption: {train_energy_joules:.2f} Joules ({train_energy_kwh:.6f} kWh)")

# ---------------------------------------------------------
# 6) INFERENCE - Monitor GPU power
# ---------------------------------------------------------

# Before starting inference power monitoring
clear_log_file(inference_power_log)

inference_power_log = "inference_power_log.csv"
print("Starting inference and logging power consumption...")
monitor_proc_infer = monitor_power(inference_power_log)
time.sleep(1)  # give nvidia-smi time to start logging

model.eval()
with torch.no_grad():
    # artificially enlarge the inference workload
    for _ in range(1):
        predictions = model(test_X.to(device))

time.sleep(1)  # ensure last logs are written
monitor_proc_infer.terminate()
monitor_proc_infer.wait()

inference_energy_joules, inference_energy_kwh = calculate_energy_consumption(inference_power_log)
print("Inference Completed!")
print(f"Inference Energy Consumption: {inference_energy_joules:.2f} Joules ({inference_energy_kwh:.6f} kWh)")

# ---------------------------------------------------------
# 7) Print out all final metrics, exactly as in the original script
# ---------------------------------------------------------
predictions_np = predictions.cpu().numpy()
test_y_np = test_y.cpu().numpy()

# Inverse transform
pred_unnorm = scaler.inverse_transform(predictions_np)
test_y_unnorm = scaler.inverse_transform(test_y_np)

# Compute MSE, RMSE, MAE on unnormalized data
mse_unnorm = mean_squared_error(test_y_unnorm, pred_unnorm)
rmse_unnorm = np.sqrt(mse_unnorm)
mae_unnorm = mean_absolute_error(test_y_unnorm, pred_unnorm)

print("\nFinal Unnormalized Metrics (on last inference results):")
print(f" MSE:  {mse_unnorm:.6f}")
print(f" RMSE: {rmse_unnorm:.6f}")
print(f" MAE:  {mae_unnorm:.6f}")
print(f"Best Validation Loss (MSE): {best_val_loss:.6f}")


<ipython-input-14-e3e87ea288f1>:46: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dataset.index = pd.to_datetime(dataset.index)


Using device: cuda
Trainable parameters: 17249
Cleared existing log file: training_power_log.csv
Starting training and logging power consumption...
Epoch 1/60, Train Loss: 0.035255, Val Loss: 0.001947
Epoch 2/60, Train Loss: 0.004710, Val Loss: 0.001040
Epoch 3/60, Train Loss: 0.003953, Val Loss: 0.001456
Epoch 4/60, Train Loss: 0.003676, Val Loss: 0.000967
Epoch 5/60, Train Loss: 0.003463, Val Loss: 0.001253
Epoch 6/60, Train Loss: 0.003220, Val Loss: 0.000589
Epoch 7/60, Train Loss: 0.003088, Val Loss: 0.000682
Epoch 8/60, Train Loss: 0.003010, Val Loss: 0.001288
Epoch 9/60, Train Loss: 0.002982, Val Loss: 0.001054
Epoch 10/60, Train Loss: 0.002844, Val Loss: 0.001426
Epoch 11/60, Train Loss: 0.002870, Val Loss: 0.000614
Epoch 12/60, Train Loss: 0.002873, Val Loss: 0.000469
Epoch 13/60, Train Loss: 0.002767, Val Loss: 0.000624
Epoch 14/60, Train Loss: 0.002811, Val Loss: 0.000505
Epoch 15/60, Train Loss: 0.002745, Val Loss: 0.000707
Epoch 16/60, Train Loss: 0.002712, Val Loss: 0.0006

ValueError: non-broadcastable output operand with shape (5253,1) doesn't match the broadcast shape (5253,6)